In [13]:
from common import *

cursor = connect()

# BEFORE DELETE

语法
```
CREATE TRIGGER trigger_name
BEFORE DELETE ON table_name
FOR EACH ROW
EXECUTE FUNCTION trigger_function_name();
```

In [9]:
sql = """
CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    price NUMERIC(10, 2) NOT NULL,
    status BOOLEAN NOT NULL DEFAULT true
);

INSERT INTO products (name, price, status) 
VALUES
  ('A', 10.99, true),
  ('B', 20.49, false),
  ('C', 15.79, true)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18d966ee8d0>

In [10]:
sql = """
CREATE OR REPLACE FUNCTION fn_before_delete_product()
RETURNS TRIGGER 
AS
$$
BEGIN
    RAISE EXCEPTION 'Deletion from the products table is not allowed.';
END;
$$
LANGUAGE plpgsql;

CREATE TRIGGER before_delete_product_trigger
BEFORE DELETE ON products
FOR EACH ROW
EXECUTE FUNCTION fn_before_delete_product();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18d966ee8d0>

In [11]:
sql = """
DELETE FROM products
WHERE id = 1;
"""
try:
    cursor.execute(sql)
except Exception as e:
    print(e)

Deletion from the products table is not allowed.
CONTEXT:  在RAISE的第3行的PL/pgSQL函数fn_before_delete_product()


# AFTER DELETE

语法
```
CREATE TRIGGER trigger_name
AFTER DELETE ON table_name
FOR EACH ROW
EXECUTE FUNCTION trigger_function_name();
```

In [14]:
sql = """
CREATE TABLE employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    salary NUMERIC(10, 2) NOT NULL
);

INSERT INTO employees(name, salary)
VALUES
   ('John Doe', 90000),
   ('Jane Doe', 80000)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18d96eddcd0>

In [15]:
sql = """
CREATE TABLE employee_archives(
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,

    salary NUMERIC(10, 2) NOT NULL,
    deleted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE OR REPLACE FUNCTION archive_deleted_employee()
RETURNS TRIGGER 
AS
$$
BEGIN
    INSERT INTO employee_archives(id, name, salary)
    VALUES (OLD.id, OLD.name, OLD.salary);
    
    RETURN OLD; 
END;
$$
LANGUAGE plpgsql;

CREATE TRIGGER after_delete_employee_trigger
AFTER DELETE ON employees
FOR EACH ROW
EXECUTE FUNCTION archive_deleted_employee();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18d96eddcd0>

In [16]:
sql = """
DELETE FROM employees 
WHERE id = 1
RETURNING *;
"""
run_sql(cursor, sql)

   id      name    salary
0   1  John Doe  90000.00


In [17]:
sql = """
SELECT * FROM employee_archives;
"""
run_sql(cursor, sql)

   id      name    salary                 deleted_at
0   1  John Doe  90000.00 2024-04-23 08:10:41.936840
